In [13]:
import win32evtlog

server = 'localhost'
logtype = 'Security'
hand = win32evtlog.OpenEventLog(server, logtype)
flags = win32evtlog.EVENTLOG_BACKWARDS_READ | win32evtlog.EVENTLOG_SEQUENTIAL_READ
total = win32evtlog.GetNumberOfEventLogRecords(hand)

while True:
    stop = 0
    events = win32evtlog.ReadEventLog(hand, flags, 0)
    if not events:
        break
    for event in events:
        print('Event Category:', event.EventCategory)
        print('Time Generated:', event.TimeGenerated)
        print('Source Name:', event.SourceName)
        print('Event ID:', event.EventID)
        print('Event Type:', event.EventType)
        print('Event String:', event.StringInserts)
        print()
        if event.TimeGenerated.strftime('%Y-%m-%d %H:%M:%S') == '2023-05-01 10:38:38':
            stop = 1
            break
        
    if stop == 1:
        break
  

Event Category: 12548
Time Generated: 2023-05-01 12:29:19
Source Name: Microsoft-Windows-Security-Auditing
Event ID: 4672
Event Type: 8
Event String: ('S-1-5-18', 'SYSTEM', 'NT AUTHORITY', '0x3e7', 'SeAssignPrimaryTokenPrivilege\r\n\t\t\tSeTcbPrivilege\r\n\t\t\tSeSecurityPrivilege\r\n\t\t\tSeTakeOwnershipPrivilege\r\n\t\t\tSeLoadDriverPrivilege\r\n\t\t\tSeBackupPrivilege\r\n\t\t\tSeRestorePrivilege\r\n\t\t\tSeDebugPrivilege\r\n\t\t\tSeAuditPrivilege\r\n\t\t\tSeSystemEnvironmentPrivilege\r\n\t\t\tSeImpersonatePrivilege\r\n\t\t\tSeDelegateSessionUserImpersonatePrivilege')

Event Category: 12544
Time Generated: 2023-05-01 12:29:19
Source Name: Microsoft-Windows-Security-Auditing
Event ID: 4624
Event Type: 8
Event String: ('S-1-5-18', 'DESKTOP-1LIHC4I$', 'WORKGROUP', '0x3e7', 'S-1-5-18', 'SYSTEM', 'NT AUTHORITY', '0x3e7', '5', 'Advapi  ', 'Negotiate', '-', '{00000000-0000-0000-0000-000000000000}', '-', '-', '0', '0x3bc', 'C:\\Windows\\System32\\services.exe', '-', '-', '%%1833', '-', '-', 

In [1]:
import win32evtlog

In [2]:
h = win32evtlog.OpenEventLog(None, "Application")

In [3]:
flags= win32evtlog.EVENTLOG_BACKWARDS_READ|win32evtlog.EVENTLOG_SEQUENTIAL_READ
records=win32evtlog.ReadEventLog(h, flags, 0)
len(records)

5

In [12]:
import codecs
import os
import sys
import time
import traceback
import win32con
import win32evtlog
import win32evtlogutil
import winerror

#----------------------------------------------------------------------
def getAllEvents(server, logtypes, basePath):
    """
    """
    if not server:
        serverName = "localhost"
    else: 
        serverName = server
    for logtype in logtypes:
        path = os.path.join(basePath, "%s_%s_log.log" % (serverName, logtype))
        getEventLogs(server, logtype, path)

#----------------------------------------------------------------------
def getEventLogs(server, logtype, logPath):
    """
    Get the event logs from the specified machine according to the
    logtype (Example: Application) and save it to the appropriately
    named log file
    """
    print("Logging %s events" % logtype)
    log = codecs.open(logPath, encoding='utf-8', mode='w')
    line_break = '-' * 80
    
    log.write("\n%s Log of %s Events\n" % (server, logtype))
    log.write("Created: %s\n\n" % time.ctime())
    log.write("\n" + line_break + "\n")
    hand = win32evtlog.OpenEventLog(server,logtype)
    total = win32evtlog.GetNumberOfEventLogRecords(hand)
    print("Total events in %s = %s" % (logtype, total))
    flags = win32evtlog.EVENTLOG_BACKWARDS_READ|win32evtlog.EVENTLOG_SEQUENTIAL_READ
    events = win32evtlog.ReadEventLog(hand,flags,0)
    evt_dict={win32con.EVENTLOG_AUDIT_FAILURE:'EVENTLOG_AUDIT_FAILURE',
              win32con.EVENTLOG_AUDIT_SUCCESS:'EVENTLOG_AUDIT_SUCCESS',
              win32con.EVENTLOG_INFORMATION_TYPE:'EVENTLOG_INFORMATION_TYPE',
              win32con.EVENTLOG_WARNING_TYPE:'EVENTLOG_WARNING_TYPE',
              win32con.EVENTLOG_ERROR_TYPE:'EVENTLOG_ERROR_TYPE'}
    
    try:
        events=1
        while events:
            events=win32evtlog.ReadEventLog(hand,flags,0)
        
            for ev_obj in events:
                the_time = ev_obj.TimeGenerated.Format() #'12/23/99 15:54:09'
                evt_id = str(winerror.HRESULT_CODE(ev_obj.EventID))
                computer = str(ev_obj.ComputerName)
                cat = ev_obj.EventCategory
        ##        seconds=date2sec(the_time)
                record = ev_obj.RecordNumber
                msg = win32evtlogutil.SafeFormatMessage(ev_obj, logtype)
                
                source = str(ev_obj.SourceName)
                if not ev_obj.EventType in evt_dict.keys():
                    evt_type = "unknown"
                else:
                    evt_type = str(evt_dict[ev_obj.EventType])
                log.write("Event Date/Time: %s\n" % the_time)
                log.write("Event ID / Type: %s / %s\n" % (evt_id, evt_type))
                log.write("Record #%s\n" % record)
                log.write("Source: %s\n\n" % source)
                log.write(msg)
                log.write("\n\n")
                log.write(line_break)
                log.write("\n\n")
    except:
        print(traceback.print_exc(sys.exc_info()))
                
    print("Log creation finished. Location of log is %s" % logPath)


if __name__ == "__main__":
    server = None  # None = local machine
    #logTypes = ["System", "Application", "Security"]
    logTypes = ["System", "Application"]
    getAllEvents(server, logTypes, "C:/Users/Louki/Downloads/")

Logging System events
Total events in System = 16717
Log creation finished. Location of log is C:/Users/Louki/Downloads/localhost_System_log.log
Logging Application events
Total events in Application = 7645
Log creation finished. Location of log is C:/Users/Louki/Downloads/localhost_Application_log.log


In [ ]:
def readEventLog(server, log_type):
    '''
    Reads the log_type (e.g., "Application" or "System") Windows events from the
    specified server.
    '''
    begin_sec = time.time()
    begin_time = time.strftime('%H:%M:%S  ', time.localtime(begin_sec))

    seconds_per_hour = 60 * 60
    how_many_seconds_back_to_search = seconds_per_hour * number_of_hours_to_look_back

    gathered_events = []

    try:
        log_handle = win32evtlog.OpenEventLog(server, log_type)

        total = win32evtlog.GetNumberOfEventLogRecords(log_handle)
        print("Scanning through {} events on {} in {}".format(total, server, log_type))

        flags = win32evtlog.EVENTLOG_BACKWARDS_READ | win32evtlog.EVENTLOG_SEQUENTIAL_READ

        event_count = 0
        events = 1
        while events:
            events = win32evtlog.ReadEventLog(log_handle, flags, 0)
            seconds = begin_sec
            for event in events:
                the_time = event.TimeGenerated.Format()
                seconds = date2sec(the_time)
                if seconds < begin_sec - how_many_seconds_back_to_search: break

                if event.EventType == win32con.EVENTLOG_ERROR_TYPE:
                    event_count += 1
                    gathered_events.append(event)
            if seconds < begin_sec - how_many_seconds_back_to_search: break  # get out of while loop as well

        win32evtlog.CloseEventLog(log_handle)
    except:
        try:
            print(traceback.print_exc(sys.exc_info()))
        except:
            print('Exception while printing traceback')

    return gathered_events

In [ ]:
# Pseudo-code for reading Windows Events
log_handle = win32evtlog.OpenEventLog(server, log_type)

while there_are_events:
    events = win32evtlog.ReadEventLog(log_handle, flags, 0)

    # Loop through returned list and do your work

win32evtlog.CloseEventLog(log_handle)